# SI649 FINAL
## Overview 

In [10]:
import streamlit as st
import pandas as pd
import altair as alt
import plotly.express as px
import plotly.graph_objects as go
from vega_datasets import data




In [11]:
# load the data
st.set_page_config(
    page_title="Population Growth and Global Climate Change",
    page_icon="🌍",
    layout="wide",
    initial_sidebar_state="expanded")

alt.themes.enable("dark")


2024-12-15 04:00:11.926 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


ThemeRegistry.enable('dark')

In [12]:
#######################
# Load data
pop_total = pd.read_csv('pop_total.csv')
regions = pd.read_csv('regions.csv')
avg_temp = pd.read_csv('avg_temp.csv')
month_avg_temp = pd.read_csv('month_avg_temp.csv')
world = pd.read_csv('world.csv')
co_emission = pd.read_csv("co-emissions-per-capita.csv")



## Visualization 0: Population change around the world (1950-2023), Relative to 1950

In [13]:
# Convert year to integer if necessary

slider = alt.binding_range(min=1950, max=2023, step=1)
select_year = alt.param(name='select_year', bind=slider, value=1950)

world = data.world_110m.url
base = alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
    fill='lightgray',
    stroke='black'
).transform_filter(
    "datum.id != 10"  # Exclude Antarctica (id=10 in world-110m.json)
).properties(
    width=800,
    height=600
    
)
points = alt.Chart(pop_total).mark_circle(
    size=2000, color='red', opacity=0.7
).transform_calculate(
    pop_increase="(datum.pop_diff) / 1000000",
).encode(
    longitude='longitude:Q',    # Longitude for positioning
    latitude='latitude:Q',     # Latitude for positioning
    size=alt.Size('pop_increase:Q', scale=alt.Scale(range=[10, 2000], zero=False),  title='Population Increase (Million)'),
    tooltip=[
        alt.Tooltip('country:N', title='Country Code'),
        alt.Tooltip('pop_increase:Q', title='Population Increase (Million)', format=',.2f'),
        alt.Tooltip('year:Q', title='Year')
    ]
).transform_filter(
    (alt.datum.year == select_year) & (alt.datum.year != 1950)  # Dynamic filter based on slider, exclude 1950
).add_params(
    select_year  # Add slider interaction
)
    



# Append the chart to the list
combined = (base + points).properties(
    title='Population Increase by Country (1950-2023), Relative to 1950',
)

combined

MaxRowsError: The number of rows in your dataset is greater than the maximum allowed (5000).

Try enabling the VegaFusion data transformer which raises this limit by pre-evaluating data
transformations in Python.
    >> import altair as alt
    >> alt.data_transformers.enable("vegafusion")

Or, see https://altair-viz.github.io/user_guide/large_datasets.html for additional information
on how to plot large datasets.

alt.LayerChart(...)

In [14]:
vis0 =(combined)

st.altair_chart(vis0, use_container_width=True)

2024-12-15 04:00:28.325 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-15 04:00:28.455 
  command:

    streamlit run /Users/melodywang/University of Michigan Dropbox/Meng Wang/SI618/SI_618_FA_24_Files/.venv/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-12-15 04:00:28.455 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

## Visualization 1: Yearly Temperature Anomalies from 1940 to 2024 (°C),
### Relative to the 1981 to 2000 Average

In [41]:
# Define the reference line (relative temp of -0.462)
reference_temp = -0.450

# Create a selection object for interactivity
click = alt.selection_single(fields=['year'], empty='none')

# Define color logic in separate steps
# base_color = alt.Color(
#     'temp:Q',
#     scale=alt.Scale(domain=[reference_temp, avg_temp['temp'].max()], range=['blue', 'red']),
#     legend=alt.Legend(title="Temperature Anomaly (°C)", values=[-1.5, -1, -0.5, 0, 0.5, 1], orient='top') 
# )
base_color = alt.Color(
    'temp:Q',
    scale=alt.Scale(scheme='redblue', domainMid=0, reverse=True),
    legend=alt.Legend(title="Relative Temp (°C)")
)

# Define the bar chart
# Create the main bar chart
chart = alt.Chart(avg_temp).mark_bar().encode(
    x=alt.X('year:O',
            title='Year',
            axis=alt.Axis(labelExpr="datum.value % 10 == 0 ? datum.value : ''", 
                          labelAngle=0, tickCount=10, ticks=False, domain=False,grid=False)
                
    ),
    y=alt.Y('temp:Q', title='Temperature Anomaly (°C)', axis=alt.Axis(ticks=False, domain=False,grid=False)),
    color=alt.condition(
        click, alt.value('orange'), base_color  # Use base_color with legend here
    ),
    opacity=alt.condition(
        click, alt.value(1.0), alt.value(0.6)
    ),
    tooltip=[
        alt.Tooltip('year:O', title='Year'),
        alt.Tooltip('temp:Q', title='Relative Temp(°C)', format='.2f'),  # Format to 2 decimals
    ]
).add_selection(
    click
).properties(
    title="Yearly Temperature Anomalies from 1940 to 2024, Relative to the 1981 to 2000 Average"
)

# Add a reference line at the relative temperature
ref_line = alt.Chart(pd.DataFrame({'y': [reference_temp]})).mark_rule(
    color='gray', strokeDash=[4, 4]
).encode(y='y:Q')

# Combine the bar chart and reference line
final_chart = chart + ref_line

# Display the chart
final_chart


## Visualization 2: Circular heatmaps representing the monthly global temperature anomalies from 1970 to 2015

### Relative to the 1981 to 2000 Average


In [415]:

# Base pie chart for each period
base = alt.Chart(month_avg_temp).mark_arc(innerRadius=50).encode(
    theta=alt.Theta('temp_diff:Q', stack=True),  # Each month is a slice
    color=alt.Color('temp_diff:Q',
                    scale=alt.Scale(scheme='redblue', domainMid=0),  # Diverging colors
                    legend=alt.Legend(title="Temperature Anomaly (°C)")
                    ),
    order=alt.Order('angle:Q'),  # Ensure correct month order
    tooltip=['month:N', 'temp_diff:Q']
)

# Base chart with a filter for a specific period
def create_period_chart(period):
    return alt.Chart(month_avg_temp).transform_filter(
        alt.datum.period == period  # Filter for one specific period
    ).mark_arc(innerRadius=50).encode(
        theta=alt.Theta('temp_diff:Q', stack=True),  # Each month's anomaly
        color=alt.Color('temp_diff:Q',
                        scale=alt.Scale(scheme='redblue', domainMid=0, reverse=True),  # Diverging colors
                        legend=alt.Legend(title="Relative Temp(°C)")
                        ),
        order=alt.Order('angle:Q'),  # Correct month order
        tooltip=[
            alt.Tooltip('month:N', title='Month'),
            alt.Tooltip('temp_diff:Q', title='Relative Temp(°C)', format='.2f')
        ]
    ).properties(
        title=period, width=150, height=150
    )

# Generate a list of charts, one for each period
chart = [create_period_chart(period) for period in month_avg_temp['period'].unique()]

# Concatenate all charts
final_chart = alt.concat(*chart, columns=5).properties(
    title="Circular Heatmaps Representing Monthly Global Temperature Anomalies (1981 - 2024), \nRelative to the 1981 to 2000 Average",
    
)

final_chart


alt.ConcatChart(...)

## Visualization 3 Annual CO2emissions per capital and top 5 emitters.

In [295]:
min_year = co_emission['year'].min()
max_year = co_emission['year'].max()

# Define a custom color scale
color_scale = alt.Scale(scheme='redblue', domainMid=0)



# Bar chart for albums with the most views
slider = alt.binding_range(min=min_year, max=max_year, step=1, name="By ")
selector = alt.selection_point(fields=['year'], bind=slider, value=max_year)

chart = alt.Chart(regions,title=alt.Title(text="CO2 Emissions per Capita by Regions (1750-2024)", fontSize=18)
).mark_bar().encode(
    x=alt.X('emission:Q', title='Emission per Capita'),
    y=alt.Y('country:O', sort='-x', title='Region'),
    color=alt.Color('country:O', title='Region', scale=color_scale),
    tooltip=[
        alt.Tooltip('country:O', title='Region'),
        alt.Tooltip('emission:Q', title='Emission per Capita', format='.2f'),  # Format to 2 decimals
        alt.Tooltip('year:O', title='Year')
    ]
).add_params(
    selector
).transform_filter(
    selector
).properties(
    width=500,
    height=500,
).configure_view(strokeWidth=0)

chart

alt.Chart(...)

## Visualization 4: World trend of CO2 Emission ()

In [307]:
# Create a DataFrame for the shaded regions
shaded_regions_data = pd.DataFrame({
    'start': [1760, 1870],
    'end': [1840, 1914],
    'label': ['First Industrial Revolution', 'Second Industrial Revolution']
})
shaded_regions_data.start = pd.to_datetime(shaded_regions_data.start, format='%Y')
shaded_regions_data.end = pd.to_datetime(shaded_regions_data.end, format='%Y')

# Create a zoomable selection
zoom = alt.selection_interval(bind='scales', encodings=['x'])
click = alt.selection_multi(fields=['year'], empty='none')



# Define the shaded regions for the industrial revolutions
shaded_regions = alt.Chart(shaded_regions_data).mark_rect(opacity=0.3, color='gray').encode(
    x='start:T',
    x2='end:T'
).add_params(
    zoom
).add_params(
    click
)


# Define the line chart for emissions
line_chart = alt.Chart(world).mark_line().encode(
    x=alt.X('year:T', title='Year', axis=alt.Axis(labelAngle=-45, tickCount=10, ticks=False, domain=False,grid=False)),
    y=alt.Y('emission:Q', title='Emission per Capita'),
    tooltip=[
        alt.Tooltip('year:T', title='Year',format='%Y'),
        alt.Tooltip('emission:Q', title='Emission')
    ]
).add_params(
    zoom
).add_params(
    click
).properties(
    width=800,
    height=400,
    title='Worldwide Average Emission Per Capita Over Time (1750 - 2024)'
)

points = alt.Chart(world).mark_point(size=20, opacity=0.8).encode(
    x=alt.X('year:T', title='Year'),
    y=alt.Y('emission:Q'),
    tooltip=[
        alt.Tooltip('year:T', title='Year', format='%Y'),
        alt.Tooltip('emission:Q', title='Emission per Capita', format='.2f'),
    ]
)


# Combine the shaded regions and the line chart
final_chart = shaded_regions + line_chart + points
final_chart


/var/folders/yz/t21562ys7x37ts3_y63c_6fm0000gn/T/ipykernel_78974/2013487399.py:3: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use selection_point instead.
  click = alt.selection_multi(fields=['year'], empty='none')


alt.LayerChart(...)